In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.core.defchararray import add
%matplotlib inline
from sklearn.metrics import confusion_matrix
# import sklearn


import tensorflow as tf
import matplotlib.image as mpimg
from skimage.io import imread
from sklearn.metrics import classification_report
import os


In [ ]:
import cv2
import xgboost as xgb
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , Dense,Flatten
from tensorflow.keras.layers import Conv2D ,MaxPooling2D,Activation , Dropout,concatenate
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
# from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:

xgb.__version__

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib


In [ ]:
data_dir='/kaggle/input/haris-et-al-injs-data/'#'/home1/srashti.goyal/bayestar_test/Haris_CIT_injs_PE/Data_to_Kaggle/'
data_dir_qts='/kaggle/input/q-tranforms-mat-gen-scripts/'

In [ ]:
# function 2: to generate input matrix given a dataframe for XGBoost
class generate_skymaps_fm():
    def __init__(self,df):
        self.df = df
        
    def load_data(self,img_paths,THETA_g):
        df = []
        for img in img_paths:
            data = np.load(img)['data']

            data/=(data*np.sin(THETA_g).T*2*np.pi*np.pi/(400*800)).sum()
    #         data = scaler.fit_transform(data)
            df.append(data)
        df = np.asarray(df)
    #     df = df.reshape(df.shape[0],image_vector_size)
        return df

    def XGBoost_input_matrix(self,data_mode_xgb='bayestar_skymaps'):
        df=self.df
        scaler =  StandardScaler()
        my_imputer = SimpleImputer()
        img_row,img_col=400,800
        image_vector_size = img_row*img_col
        THETA,PHI=np.linspace(0,np.pi,img_row),np.linspace(0,2*np.pi,img_col)
        THETA_g,PHI_g=np.meshgrid(THETA,PHI)
        
        #Defining path variables                          
        prefix_paths= df.Lensing.values.astype(str)
        prefix_paths=prefix_paths.astype('U256')
        prefix_paths[prefix_paths=='1'] = 'Lensed/' 
        prefix_paths[prefix_paths=='0'] = 'Unlensed/'
        
        
         # Selecting the type of  generated  skymap                       
        if data_mode_xgb == 'bayestar_skymaps':
            img_0_paths=add(add(data_dir+'skymaps_bayestar/',add(prefix_paths,df.img_0.values.astype(str))),'.npz')
            img_1_paths=add(add(data_dir+'skymaps_bayestar/',add(prefix_paths,df.img_1.values.astype(str))),'.npz')
            img_0= self.load_data(img_0_paths,THETA_g)
            img_1= self.load_data(img_1_paths,THETA_g)

            prior_ra = 1/(2*np.pi)
            blu=(img_0*img_1).sum(axis=(1,2))*(np.pi/400)**2/prior_ra
            #kld=(img_0*np.log(img_0/img_1)).sum(axis=(1,2))*(np.pi/400)**2
            d2=(np.mean(img_0*img_1, axis=(1,2)))
            lsq=np.mean(np.abs(img_0-img_1),axis=(1,2))
            d3=(np.std(img_0*img_1, axis=(1,2)))
            
            df= df.assign(bayestar_skymaps_blu = blu)
            df= df.assign(bayestar_skymaps_d2 = d2)
            df= df.assign(bayestar_skymaps_lsq = lsq)
            df= df.assign(bayestar_skymaps_d3 = d3)
        elif data_mode_xgb == 'pe_skymaps':
            img_0_paths=add(add(data_dir+'PE_skymaps/',add(prefix_paths,df.img_0.values.astype(str))),'.npz')
            img_1_paths=add(add(data_dir+'PE_skymaps/',add(prefix_paths,df.img_1.values.astype(str))),'.npz')
            img_0= self.load_data(img_0_paths,THETA_g)
            img_1= self.load_data(img_1_paths,THETA_g)

            prior_ra = 1/(2*np.pi)
            blu=(img_0*img_1).sum(axis=(1,2))*(np.pi/400)**2/prior_ra
            #kld=(img_0*np.log(img_0/img_1)).sum(axis=(1,2))*(np.pi/400)**2
            d2=(np.mean(img_0*img_1, axis=(1,2)))
            lsq=np.mean(np.abs(img_0-img_1),axis=(1,2))
            d3=(np.std(img_0*img_1, axis=(1,2)))
            
            df= df.assign(pe_skymaps_blu = blu)
            df= df.assign(pe_skymaps_d2 = d2)
            df= df.assign(pe_skymaps_lsq = lsq)
            df= df.assign(pe_skymaps_d3 = d3)
            
        else :  #old data
            img_0_paths=add(add('/kaggle/input/skymaps-data-gaussian-noise/',add(prefix_paths,df.img_0.values.astype(str))),'.npz')
            img_1_paths=add(add('/kaggle/input/skymaps-data-gaussian-noise/',add(prefix_paths,df.img_1.values.astype(str))),'.npz')                    
            
            img_0= self.load_data(img_0_paths,THETA_g)
            img_1= self.load_data(img_1_paths,THETA_g)


            prior_ra = 1/(2*np.pi)
            blu=(img_0*img_1).sum(axis=(1,2))*(np.pi/400)**2/prior_ra
            #kld=(img_0*np.log(img_0/img_1)).sum(axis=(1,2))*(np.pi/400)**2
            d2=(np.mean(img_0*img_1, axis=(1,2)))
            lsq=np.mean(np.abs(img_0-img_1),axis=(1,2))
            d3=(np.std(img_0*img_1, axis=(1,2)))
            
            df= df.assign(bayestar_skymaps_blu = blu)
            df= df.assign(bayestar_skymaps_d2 = d2)
            df= df.assign(bayestar_skymaps_lsq = lsq)
            df= df.assign(bayestar_skymaps_d3 = d3)
            
                                   
            
        labels = df.Lensing.values

        del img_0 ,img_1
        
        Input_combined= np.array([blu,d2,lsq,d3]).T
        Input_combined=my_imputer.fit_transform(Input_combined)
       

        return Input_combined, labels, df
    
    def XGBoost_input_matrix_from_df(self,data_mode_xgb='bayestar_skymaps'):
        df=self.df
        scaler =  StandardScaler()
        my_imputer = SimpleImputer()
        if data_mode_xgb == 'pe_skymaps':
            blu=df['pe_skymaps_blu']
            d2=df['pe_skymaps_d2']
            lsq=df['pe_skymaps_lsq']
            d3=df['pe_skymaps_d3']
        else:
            blu=df['bayestar_skymaps_blu']
            d2=df['bayestar_skymaps_d2']
            lsq=df['bayestar_skymaps_lsq']
            d3=df['bayestar_skymaps_d3']
            
        labels=df.Lensing.values
        Input_combined= np.array([blu,d2,lsq,d3]).T
        Input_combined=my_imputer.fit_transform(Input_combined)
        return Input_combined , labels, df
    
    

In [ ]:
# #function 4: to make predictions and add to the dataframe for XGBoost, given its input matrix
# def XGB_predict(model,df,x_sky,data_mode_xgb='bayestar_skymaps'):
#     y_predict = model.predict_proba(x_sky)[:,1]
#     if data_mode_xgb=='pe_skymaps':
#         df_xgb = df.assign(xgb_pred_pe_skymaps = y_predict)
#     else:
#         df_xgb = df.assign(xgb_pred_bayestar_skymaps = y_predict)
    
#     return df_xgb
    

In [ ]:
# function 7: make ROC plots in a same figure given a list of prediction columns along with the dataframe, eg: Densnet, skymaps, combined. 
def plot_ROCs(df,logy=False,cols=['dense_H1' ,'dense_L1','dense_V1','xgb_pred_bayestar_skymaps' ,'combined_pred_bayestar_skymaps'],labels=None,ylim=0,colors=['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']
):
    #if labels==None:
    #    labels=cols
    for i,col in enumerate(cols):
        efficiency=[]
        FAR=[]
        y_test=df.Lensing.values
        size=len(y_test)
        lens_size=(y_test==1).sum()
        #np.arange(0.01,1.001,0.001)
        plt.figure(1,figsize=(10,7))

        thrs=np.hstack([10**(np.arange(-5,-1,0.1)),np.arange(0.1,1.001,0.001)])
        '''
        for thr in thrs:
            y_pred=df[col]>thr
            
            results = confusion_matrix(y_test, y_pred) 
            efficiency.append(results[1,1]/lens_size)
            FAR.append(results[0,1]/(size-lens_size))
        plt.figure(1)
        plt.plot(FAR,efficiency,'-',label=col)
        '''
        nlensed = 292
        plt.figure(1)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, df[col])
        err_roc = true_positive_rate/np.sqrt(true_positive_rate*nlensed)

        #plt.errorbar(false_positive_rate,true_positive_rate,yerr=err_roc,fmt='-',label=labels[i])
        plt.plot(false_positive_rate,true_positive_rate,'-',label=labels[i],color=colors[i])
        #plt.fill_between(false_positive_rate, true_positive_rate - err_roc, true_positive_rate + err_roc,\
        #                 color=colors[i], alpha=0.2)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        print( labels[i], 'auc = %.4f'%roc_auc)

    '''   
    plt.figure(1)
    plt.xlabel('FAR')
    plt.xscale('log')
    plt.ylim(0,1)
    plt.xlim(1e-5,1)
    plt.ylabel('efficiency')
    plt.legend()
    plt.grid()
    '''
    plt.figure(1)
    plt.xlabel('FAP')
    plt.xscale('log')
    if logy==True:
        plt.yscale('log')
    plt.ylim(ylim,1)
    plt.xlim(1e-5,1)
    plt.ylabel('Efficiency')
    plt.legend()
    plt.grid()
    plt.show()
    return FAR,efficiency,thrs,false_positive_rate, true_positive_rate, thresholds

### training

In [ ]:
# #function for training xgboost data
# import xgboost as xgb
# def train_xgboost_sky(df_train,from_df=False):
#     if from_df ==False:
#         xgb_sky_train ,xgb_sky_label_train = generate_skymaps_fm(df_train).XGBoost_input_matrix()
#     else:
#         xgb_sky_train ,xgb_sky_label_train,df_train = generate_skymaps_fm(df_train).XGBoost_input_matrix_from_df(data_mode_xgb='bayestar_skymaps')
    
#     model = xgb.XGBClassifier(objective="binary:logistic",
#                       n_estimators=110,
#                       max_depth = 6 ,scale_pos_weight = .01)
#     model.fit(X=xgb_sky_train,y=xgb_sky_label_train)
#     del xgb_sky_train ,xgb_sky_label_train
#     return model

In [ ]:
def train_combined_sky(df ,dense_model_H1,dense_model_L1,dense_model_V1,from_df=False):
    if from_df == False:
        # H1 detector
        dense_train , dense_labels = generate_resize_densenet_fm(df).DenseNet_input_matrix(det = "H1")
        df = Dense_predict_H1(dense_model_H1,df,dense_train)
        del dense_train , dense_labels

        #L1 detector
        dense_train , dense_labels = generate_resize_densenet_fm(df).DenseNet_input_matrix(det = "L1")
        df = Dense_predict_L1(dense_model_L1,df,dense_train)
        del dense_train , dense_labels

        #V1 detector
        dense_train , dense_labels = generate_resize_densenet_fm(df).DenseNet_input_matrix(det = "V1")
        df = Dense_predict_V1(dense_model_V1,df,dense_train)
        del dense_train , dense_labels
        print(df)

        #XGBoost prediction -Skymaps
        xgb_sky ,xgb_sky_labels,df = generate_skymaps_fm(df).XGBoost_input_matrix() 
        

        #Training 
        model = xgb.XGBClassifier(objective="binary:logistic")
        model.fit(X= np.c_[xgb_sky,df.dense_H1,df.dense_L1,df.dense_V1],y=xgb_sky_labels)
    else:
        #XGBoost prediction -Skymaps
        xgb_sky ,xgb_sky_labels,df = generate_skymaps_fm(df).XGBoost_input_matrix_from_df(data_mode_xgb='bayestar_skymaps') 


        #Training 
        model = xgb.XGBClassifier(objective="binary:logistic",n_estimators=190,
                      max_depth = 6,scale_pos_weight = 15,min_child_weight= 10,learning_rate=0.5)
        model.fit(X= np.c_[xgb_sky,df.dense_H1,df.dense_L1,df.dense_V1],y=xgb_sky_labels)
    del xgb_sky , xgb_sky_labels
     
    return model

def train_xgboost_dense_qts(df_train,from_df=True):
    if from_df ==True:
        x1,x2,x3,labels = df_train['dense_H1'],df_train['dense_L1'],df_train['dense_V1'],df_train.Lensing.values
    else:
        print('Dense predictions not in df')
        
    model = xgb.XGBClassifier(objective="binary:logistic",
                      n_estimators=135,
                      max_depth = 6)
    xgb_qts_train = np.c_[x1,x2,x3]
    model.fit(X=xgb_qts_train,y=labels)
    return model

def predict_xgboost_dense_qts(df,model, from_df=True):
    if from_df ==True:
        x1,x2,x3 = df['dense_H1'],df['dense_L1'],df['dense_V1']
    else:
        print('Dense predictions not in df')
    input_matrix = np.c_[x1,x2,x3]

    y_predict = model.predict_proba(input_matrix)[:,1]
    df = df.assign( xgb_dense_QTS = y_predict)
        
    return df

## Combining all the data (densed and unlensed)

In [ ]:
df_unlensed = pd.read_csv("/kaggle/input/skymaps-data-gaussian-noise/Unlensed_df_train1_with_all_features.csv")
df_unlensed = df_unlensed.drop(columns=['Unnamed: 0']).reset_index(drop = True)
df_unlensed

In [ ]:
# size = 2400
df_lensed = pd.read_csv("/kaggle/input/skymaps-data-gaussian-noise/Lensed_df_full_with_all_features.csv" )
df_lensed=df_lensed.drop(columns=['Unnamed: 0'])
df_lensed['img_0']=df_lensed['img_0'].values 
df_lensed['img_1']=df_lensed['img_1'].values 
# df_lensed=df_lensed[:size]
df_unlensed = pd.read_csv("/kaggle/input/skymaps-data-gaussian-noise/Unlensed_df_train1_with_all_features.csv")
df_unlensed = df_unlensed.drop(columns=['Unnamed: 0']).reset_index(drop = True)
df_train = pd.concat([df_lensed,df_unlensed],ignore_index = True)

# df_train=df_train.sample(frac = 1,random_state=20).reset_index(drop = True)

# df_train

# size_test_l= 400
# # size_test_ul=1498

# df_lensed = pd.read_csv("/kaggle/input/skymaps-data-gaussian-noise/Lensed_df_full_with_all_features.csv" )
# df_lensed=df_lensed.drop(columns=['Unnamed: 0'])
# df_lensed['img_0']=df_lensed['img_0'].values 
# df_lensed['img_1']=df_lensed['img_1'].values 
# df_lensed=df_lensed[-size_test_l:]

df_unlensed_test = pd.read_csv("/kaggle/input/skymaps-data-gaussian-noise/Unlensed_df_test1_with_all_features.csv" )

df_unlensed_test = df_unlensed_test.drop(columns=['Unnamed: 0']).reset_index(drop = True)
# df_test = pd.concat([df_lensed,df_unlensed_test],ignore_index = True)
# df_test=df_test.sample(frac = 1,random_state=20).reset_index(drop = True)

df_all = pd.concat([df_train,df_unlensed_test],ignore_index=True)
df_all = df_all.sample(frac = 1,random_state=20).reset_index(drop = True)
df_all

In [ ]:
df_all.to_csv('df_all.csv')

In [ ]:
#  xgb_trained = train_xgboost_sky(df_train, from_df=True) 
# xgbtrain_xgboost_dense_qts(df_train,from_df=True):

## Defining the model

In [ ]:
# def cross_val_plot(model, X ,y):
# def cv_function(x_train,y_train,model):
from sklearn.model_selection import StratifiedKFold
x1,x2,x3,y = df_train['dense_H1'],df_train['dense_L1'],df_train['dense_V1'],df_train.Lensing.values

X = np.c_[x1,x2,x3]


# X , y , df_res = generate_skymaps_fm(df_all).XGBoost_input_matrix_from_df(data_mode_xgb='bayestar_skymaps')
cv = StratifiedKFold(n_splits = 10)
classifier = xgb.XGBClassifier(objective="binary:logistic",
                      n_estimators=135,
                      max_depth = 6 )
#100  , max = .01

In [ ]:
del x1,x2,x3

In [ ]:
X.shape 

In [ ]:
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

tprs = []
aucs = []

mean_fpr = 10**np.linspace(-5,0,15)
    
fig,ax = plt.subplots()
for i ,(train,test) in enumerate(cv.split(X,y)):
    
    #save X indices as a csv
    print(i)
#     print("train")
    df1 = pd.DataFrame(train)
    df1.to_csv('train-xgbqt--'+ str(i)+'.csv')
#     print(pd.DataFrame(train))
#     print("test")
    df2 = pd.DataFrame(test)
    df2.to_csv('test-xgbqt--'+ str(i)+'.csv')
#     print(pd.DataFrame(test))
    
    classifier.fit(X[train],y[train])
    
    #save xgb model
    joblib_file = "XGBQT_"+str(i)+".pkl"  
    joblib.dump(classifier, joblib_file)
    
    viz = plot_roc_curve(classifier,X[test],y[test],name="ROC fold {}".format(i),alpha=0.3,lw=1,ax=ax)
    interp_tpr = np.interp(mean_fpr,viz.fpr,viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)
        
ax.plot([0,1],[0,1],linestyle="--",lw = 2,color="r",label="Chance",alpha=0.8)
mean_tpr = np.mean(tprs,axis = 0)
mean_tpr[-1]=1.0
mean_auc = auc(mean_fpr,mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b',label = r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc , std_auc),lw = 2,alpha=.8)
    
std_tpr = np.std(tprs,axis=0)
tprs_upper = np.minimum(mean_tpr +std_tpr,1)
tprs_lower = np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr,tprs_lower,tprs_upper,color="grey",alpha=.2,label=r'$\pm$ 1 std. dev.')
    
ax.set(xlim=[-0.05,1.05],ylim=[-0.05,1.05],
          title = "ROC - XGBQT Error Bars",xscale='log',yscale='log')
ax.legend(loc ="lower right")
plt.savefig('ROC-XGB_QT-log.png')
plt.show()
    

### testing with haris et. al dataset

In [ ]:
df_test= pd.read_csv('/kaggle/input/haris-et-al-with-all-features/Predictions_saved models haris et al 1.csv')

In [ ]:
df_test["dense_H1"]

In [ ]:
# xgb_sky ,xgb_sky_labels,df_test = generate_skymaps_fm(df_test).XGBoost_input_matrix_from_df(data_mode_xgb='bayestar_skymaps') # change to load input from dataframe
# df_test = XGB_predict(xgb_trained,df_test,xgb_sky,data_mode_xgb='bayestar_skymaps')

# xgb_sky ,xgb_sky_labels,df_test = generate_skymaps_fm(df_test).XGBoost_input_matrix_from_df(data_mode_xgb='pe_skymaps') # change to load input from dataframe

# df_test = XGB_predict(xgb_trained,df_test,xgb_sky,data_mode_xgb='pe_skymaps')
df_test



In [ ]:
for i in range(10):
    xgb_sky ,xgb_sky_labels,df_test = generate_skymaps_fm(df_test).XGBoost_input_matrix_from_df(data_mode_xgb='bayestar_skymaps') # change to load input from dataframe
    xgb_cv = joblib.load('./XGBsky_'+str(i)+'.pkl')
    df = XGB_predict(xgb_cv,df_test,xgb_sky,data_mode_xgb='bayestar_skymaps')
    df_test['xgb_pred_bayestar_skymaps_' +str(i) ]=df['xgb_pred_bayestar_skymaps']

    xgb_sky ,xgb_sky_labels,df_test = generate_skymaps_fm(df_test).XGBoost_input_matrix_from_df(data_mode_xgb='pe_skymaps') # change to load input from dataframe
    df = XGB_predict(xgb_cv,df_test,xgb_sky,data_mode_xgb='pe_skymaps')
    df_test['xgb_pred_pe_skymaps_' +str(i) ]=df['xgb_pred_pe_skymaps']

    df_test

In [ ]:
scaler =  StandardScaler()
my_imputer = SimpleImputer()
df_test[['bayestar_skymaps_blu_scaled','pe_skymaps_blu_scaled']]=my_imputer.fit_transform(df_test[['bayestar_skymaps_blu','pe_skymaps_blu']])

In [ ]:
FAR,efficiency,thrs,false_positive_rate, true_positive_rate, thresholds=plot_ROCs(df_test,cols=['xgb_pred_pe_skymaps','xgb_pred_bayestar_skymaps','bayestar_skymaps_blu_scaled','pe_skymaps_blu_scaled','BLU_ra_dec'],\
                                                                                 labels=['ML PE skymaps','ML Bayestar skymaps','$B^L_U$ Bayestar skymaps','$B^L_U$ PE skymaps','$B^L_U$ : RA DEC'])

In [ ]:
cols=['xgb_pred_pe_skymaps','xgb_pred_bayestar_skymaps','bayestar_skymaps_blu_scaled','pe_skymaps_blu_scaled','BLU_ra_dec']
labels=['ML PE skymaps','ML Bayestar skymaps','$B^L_U$ Bayestar skymaps','$B^L_U$ PE skymaps','$B^L_U$ : RA DEC']
idx =np.arange(10).astype(str)
c=add('xgb_pred_bayestar_skymaps_', idx)
cols=np.append(c,cols)
d=add('ML bayestar CV ', idx)
labels=np.append(d,labels)
colors=['k','k','k','k','k','k','k','k','k','k','C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']

FAR,efficiency,thrs,false_positive_rate, true_positive_rate, thresholds=plot_ROCs(df_test,cols=cols,labels=labels,colors=colors)

In [ ]:
FAR,efficiency,thrs,false_positive_rate, true_positive_rate, thresholds=plot_ROCs(df_test,cols=cols,labels=labels,colors=colors,logy=True)